In [26]:
using Revise

using Test
import TensorCrossInterpolation as TCI
import Random

In [27]:
using Test
import TensorCrossInterpolation as TCI
import TensorCrossInterpolation: rank, linkdims, TensorCI2, MultiIndex, evaluate, crossinterpolate2, pivoterror, tensortrain
import Random
import QuanticsGrids as QD

In [28]:
import TensorCrossInterpolation as TCI
using TensorCrossInterpolation
using Random
using Test
using ITensors
ITensors.disable_warn_order()

In [29]:
pivotsearch = :full
seed = 1234567

Random.seed!(seed)

#==
R = 30
abstol = 1e-1
grid = QD.DiscretizedGrid{1}(R, (0.0,), (1.0,))

f(bitlist) = fx(QD.quantics_to_origcoord(grid, bitlist)[1])

B = 2^(-30) # global variable

nc = 1000
coeffs = [randn() for _ in 1:nc]
exps = 10 .* rand(nc)

function fx(x)
    #return cos(x / B) * cos(x / (4 * sqrt(5) * B)) * exp(-x^2) + 2 * exp(-x)
    #cos(x^2) + exp(-x^2) + 2 * exp(-x)
    sum(c * exp(- e * x^2) for (c, e) in zip(coeffs, exps))
end
==#

R = 20
abstol = 1e-3
sites = [Index(2, "n=$n") for n in 1:R]

Ψ = random_mps(sites; linkdims=10)

tensor = Array(reduce(*, Ψ), reverse(sites))
noise = (0.3 * abstol) .* randn(size(tensor))
@show sum(abs.(noise) .> abstol)
tensor .+= noise

f(x) = tensor[x...]

localdims = fill(2, R)
firstpivot = ones(Int, R)
tci, ranks, errors = crossinterpolate2(
    Float64,
    f,
    localdims,
    [firstpivot];
    tolerance=abstol,
    maxbonddim=typemax(Int),
    maxiter=typemax(Int),
    loginterval=1,
    verbosity=1,
    normalizeerror=false,
    pivotsearch=pivotsearch,
)
@show abstol

sum(abs.(noise) .> abstol) = 878
(bondindex, length(Iset)) = (1, 2)
(bondindex, length(Iset)) = (2, 2)
(bondindex, length(Iset)) = (3, 1)
(bondindex, length(Iset)) = (4, 1)
(bondindex, length(Iset)) = (5, 1)
(bondindex, length(Iset)) = (6, 2)
(bondindex, length(Iset)) = (7, 1)
(bondindex, length(Iset)) = (8, 1)
(bondindex, length(Iset)) = (9, 1)
(bondindex, length(Iset)) = (10, 1)
(bondindex, length(Iset)) = (11, 1)
(bondindex, length(Iset)) = (12, 1)
(bondindex, length(Iset)) = (13, 1)
(bondindex, length(Iset)) = (14, 1)
(bondindex, length(Iset)) = (15, 1)
(bondindex, length(Iset)) = (16, 1)
(bondindex, length(Iset)) = (17, 1)
(bondindex, length(Iset)) = (18, 1)
(bondindex, length(Iset)) = (19, 1)
iteration = 1, rank = 10, error= 0.000752041844966517, maxsamplevalue= 0.003443031638096202, nglobalpivot=0
(bondindex, length(Iset)) = (1, 2)
(bondindex, length(Iset)) = (2, 4)
(bondindex, length(Iset)) = (3, 5)
(bondindex, length(Iset)) = (4, 8)
(bondindex, length(Iset)) = (5, 6)
(bondinde

0.001

In [30]:
#@show length(tci.globalpivots)
#@show length(TCI.reducedglobalpivots(tci.globalpivots))
#@show length(TCI.fullglobalpivots(TCI.reducedglobalpivots(tci.globalpivots)))

In [31]:
for b in 1:length(tci)-1
    @show length(TCI.Iset(tci, b+1)), length(TCI.Jset(tci, b))
end

(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (2, 2977)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (4, 2969)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (8, 2951)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (16, 2916)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (32, 2841)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (64, 2708)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (128, 2499)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (256, 2144)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (512, 1639)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (1024, 1024)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (1587, 512)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (2061, 256)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (2455, 128)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (2700, 64)
(length(TCI.Ise

In [37]:
#for orthocenter in [1, length(tci), length(tci) ÷ 2]
for orthocenter in [1]
    tt = TCI.TensorTrain(TCI.sitetensors(tci, f; orthocenter=orthocenter))
    TCI.compress!(tt, :SVD; tolerance=1e-8)
    tt = TCI.TTCache(tt)
    println("Orthocenter: ", orthocenter)
    @show TCI.linkdims(tt)
    println("Error on pivots:")
    for b in 1:length(tci)-1
        Iset = TCI.Iset(tci, b+1)
        Jset = TCI.Jset(tci, b)
        diff = maximum(abs, [TCI.evaluate(tt, vcat(i, j)) - f(vcat(i, j)) for i in Iset, j in Jset])
        println(b, "    ", diff, " ", diff > abstol)
    end
    println("Error on T:")
    for l in 1:length(tci)-1
        diff = [TCI.evaluate(tt, vcat(i, m, j)) - f(vcat(i, m, j)) for i in TCI.Iset(tci, l), j in TCI.Jset(tci, l), m in 1:localdims[l]]
        println(l, "    ", maximum(abs, diff), " ", maximum(abs, diff) > abstol)
    end

    println("Error on Π:")
    for b in 1:length(tci)-1
        Iset_b = TCI.Iset(tci, b)
        Jset_bp1 = TCI.Jset(tci, b+1)
        diff = [
            TCI.evaluate(tt, vcat(i, i1, i2, j)) - f(vcat(i, i1, i2, j)) for i in Iset_b, j in Jset_bp1, i1 in 1:localdims[b], i2 in 1:localdims[b+1]]
        println(b, "    ", maximum(abs, diff), " ", maximum(abs, diff) > abstol)
    end

    println("error on global pivots: ", maximum([abs(TCI.evaluate(tt, p) - f(p)) for p in tci.globalpivots]))
end

Orthocenter: 1
TCI.linkdims(tt) = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 512, 256, 128, 64, 32, 16, 8, 4, 2]
Error on pivots:
1    2.1337098754514727e-16 false
2    2.1337098754514727e-16 false
3    2.1337098754514727e-16 false
4    2.1337098754514727e-16 false
5    2.1337098754514727e-16 false
6    2.1337098754514727e-16 false
7    2.1337098754514727e-16 false
8    2.1337098754514727e-16 false
9    2.220446049250313e-16 false
10    2.220446049250313e-16 false
11    2.220446049250313e-16 false
12    2.220446049250313e-16 false
13    2.1337098754514727e-16 false
14    2.1337098754514727e-16 false
15    2.1337098754514727e-16 false
16    2.1337098754514727e-16 false
17    2.1337098754514727e-16 false
18    2.1337098754514727e-16 false
19    2.1337098754514727e-16 false
Error on T:
1    2.1337098754514727e-16 false
2    2.1337098754514727e-16 false
3    2.1337098754514727e-16 false
4    2.1337098754514727e-16 false
5    2.1337098754514727e-16 false
6    2.1337098754514727e-16 false
7 

InterruptException: InterruptException: